# MeanShift and CAMShift

+ MeanShift can be given a target to track, calculate the color histogram of the target area, and then keep sliding the tracking window to the closest match (the cluster center).
+ Just using MeanShift won't change the windwo size if the target moves away or towards the camera.
+ We can use CAMShift to update the size of the window.


In [ ]:
import numpy as np
import cv2

In [ ]:
capture = cv2.VideoCapture(0)

ret,frame = capture.read()


face_cascade = cv2.CascadeClassifier("haarcascades/haarcascade_frontalface_default.xml")
face_rects = face_cascade.detectMultiScale(frame)
print(face_rects)

(face_x,face_y,w,h) = tuple(face_rects[0])
track_window = (face_x,face_y,w,h)

roi = frame[face_y,:face_y+h,face_x,face_x+w]

hsv_roi = cv2.cvtColor(roi,cv2.COLOR_BGR2HSV)

roi_hist = cv2.calcHist([hsv_roi],[0],None,[180],[0,180])

cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)

term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10,1)

while True:
    
    ret,frame = capture.read()
    
    if ret == True:
        hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
        
        dst = cv2.calcBackProject([hst],[0],roi_hist,[0,180],1)
        
        ret,track_window = cv2.meanShift(dst,track_window,term_crit)
        
        x,y,w,h = track_window
        img2 = cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),5)
        
        cv2.imshow("Image",img2)
        
        k = cv2.waitKey(1) & 0xFF
        if k == 27:
            break
    else:
        break
        
        
cv2.destroyAllWindows()
capture.release()

In [ ]:
####################################
ret, track_window = cv2.CamShift(dst,track_window,term_crit)

pts = cv2.boxPoints(ret)
pts = np.int0(pts)
img2 = cv2.polylines(frame,[pts],True,(0,0,255),5)

####################################